        Prototyping notebook for predicting stock volaitility, prices, etc using extra data from web trends, news, etc. 

python packages needed: compress_pickle[lz4] pandas numpy yahooquery seaborn

If for some reason you wish to download the data yourself, you can do so by following these steps:

1) Download and extract https://www.kaggle.com/datasets/footballjoe789/us-stock-dataset

2) From the stock dataset, copy "Stock_List.csv" and "Stocks/*" to "data/". You can delete the other files as they are not used (at least yet).

In [1]:
try:
    import pandas as pd
    # set pandas to use pyarrow')
    pd.set_option('io.parquet.engine', 'pyarrow')
    import numpy as np
    import seaborn
    from glob import glob
    from compress_pickle import dump, load
    import os
    from yahooquery import Ticker
    import timeit
    import time
    import gc, os
    import datetime
    from getpass import getpass
    from shutil import rmtree
    from utils import fernet_key_encryption, aquire_stock_search_terms as aquire_terms, get_macroeconomic_data as macro_data, download_datasets, load_file, interpolate_months_to_days, intersect_df
except ModuleNotFoundError as e:
    print(e)
    print('Please install the missing module(s)')
    print("pip install compress_pickle[lz4] pandas numpy yahooquery cryptography seaborn kaggle pyarrow")
    


In [2]:
# After this first run, you will only need to enter the password to load the api keys.
# If you need to change the keys or password, delete the relevent .secret keys file and run this section again.
# salt.secret is a non-sensitive file that is used to both generate the encryption key as well as decryption. If this key is lost, the encrypted files are lost and you will need to re-enter the api keys.

# Ask for input of password to save API keys
password = getpass("Enter password to save/load API keys: ");

kaggle_api_key = fernet_key_encryption(password, 'Kaggle');
#td_ameritrade_api_key = fernet_key_encryption(password, 'TD_Ameritrade')

#data.nasdaq.com api key
data_nasdaq_key = fernet_key_encryption(password, 'Nasdaq');

del password;
gc.collect();


In [3]:
username, password = kaggle_api_key.split(' ');
os.environ['KAGGLE_USERNAME'] = username;
os.environ['KAGGLE_KEY'] = password;
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd();

# download the various kaggle datasets
download_datasets(
        'https://www.kaggle.com/datasets/sarthmirashi07/us-macroeconomic-data', 
        kaggle_api_key, 
        files_to_move={'US_macroeconomics.csv': 'macro/US_macroeconomics.csv'},
        delete=True,
        dest_name='Macro')

download_datasets(
        'https://www.kaggle.com/datasets/footballjoe789/us-stock-dataset', 
        kaggle_api_key, 
        files_to_move={'us-stock-dataset/Stock_List.csv': 'Stock_List.csv', 'us-stock-dataset/Data/Stocks': 'Stocks'}, 
        delete=True,
        dest_name='Stocks')


# clear the username and key from the environment variables
os.environ['KAGGLE_USERNAME'] = "" 
os.environ['KAGGLE_KEY'] = ""

Dataset (Macro) already downloaded.
Dataset (Stocks) already downloaded.


In [4]:
# Gather the company info for all the ticker symbols and return a dataframe with relevant search terms for each company.
# If the stocks dataset is updated on kaggle, compank_list.pkl needs to be deleted and this run again if the symbols have changed. It would be more efficient to manually pull the new data ourselves.
search_terms = aquire_terms('data/Stocks/')
search_terms.data

,Ticker,Company,Top_Executive,Industry,quoteType,sector,first_traded,year_founded
0,A,"Agilent Technologies, Inc.",Michael McMullen,Diagnostics & Research,EQUITY,Healthcare,1999-11-18 08:30:00,1999
1,AA,Alcoa Corporation,Roy Harvey,Aluminum,EQUITY,Basic Materials,-252322200,1886
2,AAC,Ares Acquisition Corporation,David Kaplan,Shell Companies,EQUITY,Financial Services,2021-03-25 08:30:00,2020
3,AACG,ATA Creativity Global,Xiaofeng Ma,Education & Training Services,EQUITY,Consumer Defensive,2008-01-29 08:30:00,1999
4,AACI,Armada Acquisition Corp. I,Stephen Herbert,Shell Companies,EQUITY,Financial Services,2021-11-10 08:30:00,2020
...,...,...,...,...,...,...,...,...
7528,ZVSA,None,Stephen Glover,Biotechnology,EQUITY,Healthcare,2022-02-11 08:30:00,
7529,ZWS,Zurn Elkay Water Solutions Corporation,Todd Adams,Pollution & Treatment Controls,EQUITY,Industrials,2012-03-29 08:30:00,2006
7530,ZYME,Zymeworks Inc.,Ali Tehrani PhD,Biotechnology,EQUITY,Healthcare,2017-04-28 08:30:00,2003
7531,ZYNE,"Zynerba Pharmaceuticals, Inc.",Armando Anido MBA,Drug Manufacturers—Specialty & Generic,EQUITY,Healthcare,2015-08-05 08:30:00,2007


In [5]:
# gether historical macroeconomic data from different sources.
#macro_data = macro_data('data/Macro/')

df1 = load_file('data/Macro/US_macroeconomics.parquet')
df2 = load_file('data\Stocks\AAPL.parquet')

#for col in [x for x in df.columns if x not in ["CPI", "date", "Unemp_rate", "mortgage_rate"]]:
    #df[col] = df[col].div(df['CPI'])
    


df1, df2 = intersect_df(df1, df2, True)
df1


AttributeError: 'DataFrame' object has no attribute 'date'

In [6]:
df2

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1980-12-12 00:00:00-05:00,0.099722,0.100155,0.099722,0.099722,469033600,0.0,0.0
1,1980-12-15 00:00:00-05:00,0.094953,0.094953,0.094519,0.094519,175884800,0.0,0.0
2,1980-12-16 00:00:00-05:00,0.088015,0.088015,0.087581,0.087581,105728000,0.0,0.0
3,1980-12-17 00:00:00-05:00,0.089749,0.090183,0.089749,0.089749,86441600,0.0,0.0
4,1980-12-18 00:00:00-05:00,0.092351,0.092785,0.092351,0.092351,73449600,0.0,0.0
...,...,...,...,...,...,...,...,...
10636,2023-02-21 00:00:00-05:00,150.199997,151.300003,148.410004,148.479996,58867200,0.0,0.0
10637,2023-02-22 00:00:00-05:00,148.869995,149.949997,147.160004,148.910004,51011300,0.0,0.0
10638,2023-02-23 00:00:00-05:00,150.089996,150.339996,147.240005,149.399994,48394200,0.0,0.0
10639,2023-02-24 00:00:00-05:00,147.110001,147.190002,145.720001,146.710007,55418200,0.0,0.0
